In [10]:
# imports
from nett import Brain, Body, Environment
from nett import NETT

### Encoder

An encoder is part of the Brain component, and generates a feature representation (embedding) for the incoming observations. Specifically, for each set of observations, the encoder consumes them to produce an N-dimensional vector that is passed on to the `Policy` Network to produce a prediction (the action that the body will take). 

![title](/home/desabh/netts-tutorials/Encoder.png)

In principle, the encoder can be anything as long as it can take the observation set produced from the environment and produce an embedding vector. 

![title](/home/desabh/netts-tutorials/Encoder-Detail.png)

In the following notebook, we will define a custom encoder which is a `Convolutional Neural Network` (CNN) that produces an embedding vector of `256`. 

In [8]:
import torch

from gym import spaces
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor

A custom Encoder MUST inherit from the `BaseFeaturesExtractor` class from `stable_baselines3`. 

In [9]:
class CustomCNN(BaseFeaturesExtractor):
    """
    :param observation_space: (gym.Space)
    :param features_dim: (int) Number of features extracted.
        This corresponds to the number of unit for the last layer.
    """

    def __init__(self, observation_space: spaces.Box, features_dim: int = 256):
        super().__init__(observation_space, features_dim)
        # We assume CxHxW images (channels first)
        # Re-ordering will be done by pre-preprocessing or wrapper
        n_input_channels = observation_space.shape[0]
        self.cnn = torch.nn.Sequential(
            torch.nn.Conv2d(n_input_channels, 32, kernel_size=8, stride=4, padding=0),
            torch.nn.ReLU(),
            torch.nn.Conv2d(32, 64, kernel_size=4, stride=2, padding=0),
            torch.nn.ReLU(),
            torch.nn.Flatten(),
        )

        # Compute shape by doing one forward pass
        with torch.no_grad():
            n_flatten = self.cnn(
                torch.as_tensor(observation_space.sample()[None]).float()
            ).shape[1]

        self.linear = torch.nn.Sequential(torch.nn.Linear(n_flatten, features_dim), torch.nn.ReLU())

    def forward(self, observations: torch.Tensor) -> torch.Tensor:
        return self.linear(self.cnn(observations))

#### Define the components

In [ ]:
# define components (brain, body and environment)
# this example shows only a minimal setup, a LOT of customization is possible here
brain = Brain(encoder=CustomCNN, train_encoder=True)
body = Body(type="basic", dvs=False)
environment = Environment(config="parsing", executable_path="/home/desabh/builds/parsing/parsing-linux.x86_64")

#### Run

In [ ]:
# construct the NETT
nett = NETT(brain=brain, body=body, environment=environment)
# run the NETT
jobs = nett.run(dir="./parsing_testrun", num_brains=5, train_eps=1000, test_eps=20)

#### Analyze

In [ ]:
nett.analyze(run_dir="./parsing_testrun", output_dir="./analysis_results", ep_bucket=100, num_episodes=1000)